<h3> Function to calculate Balassa index of patent radicality </h3>

<p> Matt Wilder, University of Toronto <br>
Please address questions and comments to <a href="mailto:matt.wilder@utoronto.ca">matt.wilder@utoronto.ca</a>. </p>

<p>Updated 24 April 2024</p>

This function calculates the Balassa Index of patent radicality, as used as a measure of revealed comparative advantage (RCA) in Meelan et al (2017) 'Disentangling patterns of economic, technological and innovative specialization of Western economies: An assessment of the Varieties-of-Capitalism theory on comparative institutional advantages' <i>Research Policy, 46</i>(3): 667-77.

The Radicality Index (RI) is defined as follows:
$$
\large RI_{ijt} = \frac{\frac{Y^*_{ijt}}{(Y_{ijt} + Y^*_{ijt})}}{\sum_{i=1}^{n} \frac{Y^*_{ijt}}{(Y_{ijt} + Y^*_{ijt})}}
$$
where:

$RI_{ijt}$ is the Radicality Index for country $i$ in sector $j$ at time $t$.<br>
$Y^*_{ijt}$ denotes the number of radical patents for country $i$ in sector $j$ at time $t$.<br>
$Y_{ijt}$ refers to the number of non-radical patents of country $i$ in sector $j$ at time $t$. <br>
The denominator is the sum over all countries of their individual ratios of radical to total patents in sector $j$ at time $t$.



This index captures the relative specialization of countries in radical innovation within specific sectors. Technically, the range of the variable is 0 to $\infty$. Conventional interpretation is as follows:
<ul>
<li>RI > 1: the country has a revealed comparative advantage in the sector. </li>
<li>RI < 1: the country has a revealed comparative disadvantage in the sector.</li>
</ul>

Meelan et al (2017) normalize the distrubution between -1 and 1, whereby values below zero signify a comparative disadvantage, while values above zero signify a comparative advantage. 
$$
\normalsize RI' = \frac{RI - 1}{RI + 1}
$$
<ul>
<li>RI' > 0: the country has a revealed comparative advantage in the sector. </li>
<li>RI' < 0: the country has a revealed comparative disadvantage in the sector. </li>
</ul>



In [1]:
import pandas as pd
import numpy as np
import ast


'''load in the sample data (this may take a few minutes if the connection is slow)'''

df = pd.read_csv('https://raw.githubusercontent.com/matt-wilder/patent-research/main/forward_citations_with_cpc_classifications_sample.csv',sep='\t', dtype='str')
df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.0,NaN,US,LME
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.0,NaN,JP,CME
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.0,NaN,US,LME
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME


In [2]:
''' define the tech class '''

'''by the section codes (8 unique in this dataset)'''
df['tech_class'] = df['patent_cpcs'].str.split('|').str[0].str[0]

# A - Human Necessities (e.g., Health, Food)
# B - Performing Operations; Transporting (e.g., Manufacturing, Vehicles)
# C - Chemistry; Metallurgy (e.g., Chemicals, Alloys)
# D - Textiles; Paper (e.g., Fabrics, Papermaking)
# E - Fixed Constructions (e.g., Buildings, Infrastructure)
# F - Mechanical Engineering; Lighting; Heating; Weapons; Blasting (e.g., Machines, HVAC)
# G - Physics (e.g., Optics, Instruments)
# H - Electricity (e.g., Circuits, Communications)

'''or by the classification codes (82 unqiue classes in this dataset)'''
# df['tech_class'] = df['patent_cpcs'].str.split('|').str[0].str[:3]

'''or using fractional counting'''
def process_patent(row):
    if pd.notna(row['patent_cpcs']):  # Check if 'patent_cpcs' is not NaN
        cpcs = row['patent_cpcs'].split('|')

        # count occurrences of each CPC
        cpc_counts = {}
        for cpc in set(cpcs):
            cpc_counts[cpc] = cpcs.count(cpc)

        # Calculate fractional contributions for each CPC, rounding to two decimal places
        total_cpcs = len(cpcs)
        cpc_contributions = {cpc: round(count / total_cpcs, 2) for cpc, count in cpc_counts.items()}

        return cpc_contributions
    else:
        return {}
df['frac_class'] = df.apply(process_patent, axis=1)

df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC,tech_class,frac_class
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME,C,{'C25B': 1.0}
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME,G,"{'G03B': 0.5, 'H04N': 0.5}"
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME,H,{'H02K': 1.0}
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME,Y,"{'Y10S': 0.5, 'H01L': 0.5}"
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME,G,"{'G01M': 0.5, 'Y10T': 0.5}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.0,NaN,US,LME,NaN,{}
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.0,NaN,JP,CME,NaN,{}
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.0,NaN,US,LME,NaN,{}
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME,NaN,{}


In [3]:
'''calculate radicality measure'''

# function to calculate deciles for radicality measures by tech class and time period (window based on the number of patents required to populate a decile distribution, per "window_" variables)
def calculate_deciles_for_patents(df, indicators, year_column='year', tech_class_column='tech_class'): #tech class based on first listed tech class, not fractional count
    # ensure df has a unique index
    df.reset_index(inplace=True, drop=True)
    
    # store the decile and time frame ('window') information for each indicator
    deciles_info = pd.DataFrame(index=df.index)

    # iterate over each indicator, calculating deciles and windows separately
    for indicator in indicators:
        # initialize a list to store windows for each indicator
        deciles_info[f'window_{indicator}'] = pd.NA
        
        # filter out NaNs and values < 0
        indicator_df = df.dropna(subset=[indicator])
        indicator_df = indicator_df[indicator_df[indicator] > 0]

        # ensure year is treated as integer
        indicator_df[year_column] = indicator_df[year_column].astype(int)

        # group by tech class and sort unique years within each tech class for each indicator
        for tech_class, group in indicator_df.groupby(tech_class_column):
            years = sorted(group[year_column].unique())
            grouped_years = []

            # process each year within the tech class
            for year in years:
                if year not in grouped_years:
                    current_group = [year]
                    while True:
                        temp_df = group[group[year_column].isin(current_group)].copy()
                        if len(temp_df) >= 10 or (min(current_group) == min(years) and max(current_group) == max(years)):
                            window = f"{min(current_group)} - {max(current_group)}"
                            grouped_years.extend(current_group)
                            
                            # calculate deciles for the current group
                            decile_label = f'{indicator}_decile'
                            temp_df[decile_label] = pd.qcut(temp_df[indicator].rank(method='first'), 10, labels=range(1, 11), duplicates='drop')
                            temp_df[f'window_{indicator}'] = window  # Create the column to store the window information
                            
                            # update the deciles_info df with decile and window info
                            deciles_info.loc[temp_df.index, decile_label] = temp_df[decile_label]
                            deciles_info.loc[temp_df.index, f'window_{indicator}'] = window
                            
                            break
                        else:
                            if min(current_group) > min(years):
                                current_group.insert(0, min(current_group) - 1)
                            if max(current_group) < max(years):
                                current_group.append(max(current_group) + 1)
                            if min(current_group) <= min(years) and max(current_group) >= max(years):
                                break

    # concatenate the original df with the deciles_info df
    result_df = pd.concat([df, deciles_info], axis=1)

    return result_df

indicators = ['forward_citations', 'generality_hhi']  # these indicators need to be converted to numeric for the calculation to work
df[['forward_citations', 'generality_hhi']] = df[['forward_citations', 'generality_hhi']].apply(pd.to_numeric, errors='coerce')

df = calculate_deciles_for_patents(df, indicators) # note the OR statement and decile specification; set these as desired
df['is_radical'] = ((df['forward_citations_decile'] == 10) | (df['generality_hhi_decile'] == 10)).astype(int)

df

,patent,inventors_country_frac,assignees_country_frac,inventors_country_mode,assignees_country_mode,name,date,year,forward_citations,generality_hhi,patent_cpcs,country,VoC,tech_class,frac_class,window_forward_citations,forward_citations_decile,window_generality_hhi,generality_hhi_decile,is_radical
0,4240895,{'US': 1.0},{},['US'],NaN,Lees,1991-02-01,1991,11.0,0.88,C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C25B|C...,US,LME,C,{'C25B': 1.0},1990 - 1991,7,1990 - 1991,9,0
1,4893178,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Matama et al.,1990-01-01,1990,11.0,0.56,G03B|H04N|G03B|H04N|G03B|H04N|G03B|H04N|G03B|H...,JP,CME,G,"{'G03B': 0.5, 'H04N': 0.5}",1990 - 1991,4,1990 - 1991,2,0
2,4896067,{'DE': 1.0},{'DE': 1.0},['DE'],['DE'],Walther,1990-01-01,1990,14.0,0.66,H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H02K|H...,DE,CME,H,{'H02K': 1.0},1990 - 1991,6,1990 - 1991,5,0
3,4900212,{'JP': 1.0},{'US': 1.0},['JP'],['US'],Mikahara,1990-02-01,1990,18.0,0.76,Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H01L|Y10S|H...,US,LME,Y,"{'Y10S': 0.5, 'H01L': 0.5}",<NA>,NaN,<NA>,NaN,0
4,4903398,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Sakamoto et al.,1990-02-01,1990,11.0,0.72,G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y10T|G01M|Y...,JP,CME,G,"{'G01M': 0.5, 'Y10T': 0.5}",1990 - 1991,4,1990 - 1991,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,D423357,{},{},NaN,NaN,Chrisco et al.,2000-04-01,2000,4.0,0.00,NaN,US,LME,NaN,{},NaN,NaN,NaN,NaN,0
496,D427997,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Inoue,2000-07-01,2000,8.0,0.00,NaN,JP,CME,NaN,{},NaN,NaN,NaN,NaN,0
497,D430278,{},{},NaN,NaN,Krauss et al.,2000-08-01,2000,14.0,0.00,NaN,US,LME,NaN,{},NaN,NaN,NaN,NaN,0
498,D433628,{'JP': 1.0},{'JP': 1.0},['JP'],['JP'],Tamura,2000-11-01,2000,5.0,0.69,NaN,JP,CME,NaN,{},NaN,NaN,NaN,NaN,0


<h3>Crude Balassa index of innovation advantage</h3>

$$
\large RI_{ijt} = \frac{\frac{Y^*_{ijt}}{(Y_{ijt} + Y^*_{ijt})}}{\sum_{i=1}^{n} \frac{Y^*_{ijt}}{(Y_{ijt} + Y^*_{ijt})}}
$$

$Y^*_{ijt} = $ number of 10<sup>th</sup> decile patents in country $i$ in tech class $j$ in year $t$.<br>
$Y_{ijt} = $ number of all other patents in country $i$ in tech class $j$ in year $t$. <br>
The denominator is the sum over all countries of their individual ratios of radical to total patents in tech class $j$ in year $t$.

In [4]:
'''calculate a Radicality (Balassa) Index for each country i in tech class j at time t'''

# Step 1: aggregate data
aggregated_data = df.groupby(['country', 'tech_class', 'year', 'is_radical']).size().unstack(fill_value=0)
aggregated_data.reset_index(inplace=True)

# Step 2: calculate RI numerator
aggregated_data['radical_fraction'] = aggregated_data[1] / (aggregated_data[0] + aggregated_data[1])
aggregated_data.rename(columns={0: 'non_radical_count', 1: 'radical_count'}, inplace=True)

# Step 3: calculate RI denominator
total_radical_fraction = aggregated_data.groupby(['tech_class', 'year'])['radical_fraction'].sum()

# make sure the denominator is never zero or NaN (replace 0 with a very small number, e.g., 1e-10, to avoid division by zero)
total_radical_fraction = total_radical_fraction.replace(0, 1e-10)

# Step 4: compute RI
aggregated_data['RI'] = aggregated_data.apply(
    lambda row: row['radical_fraction'] / total_radical_fraction.get((row['tech_class'], row['year']), 1e-10), axis=1
)

# Step 5: normalize RI using the transformation formula
aggregated_data['RI_normalized'] = (aggregated_data['RI'] - 1) / (aggregated_data['RI'] + 1)

# display the data frame of Rijt values (RI for each country, tech class, and year)
aggregated_data                    

# aggregated_data[aggregated_data['RI_normalized'] > 0]  # or display only cases of comparative advantage

is_radical,country,tech_class,year,non_radical_count,radical_count,radical_fraction,RI,RI_normalized
0,AT,D,1992,1,0,0.000,0.000000,-1.000000
1,AU,F,1993,1,0,0.000,0.000000,-1.000000
2,BE,G,1991,0,1,1.000,0.714286,-0.166667
3,CA,A,1992,0,1,1.000,1.000000,0.000000
4,CA,A,1993,1,0,0.000,0.000000,-1.000000
...,...,...,...,...,...,...,...,...
183,US,H,1996,3,2,0.400,1.000000,0.000000
184,US,H,1997,2,0,0.000,0.000000,-1.000000
185,US,H,1999,3,2,0.400,0.761905,-0.135135
186,US,H,2000,5,3,0.375,0.252336,-0.597015


<h3>Using Fractional Counts to Calculate a Balassa Index of Innovation Advantage</h3>

Fractional counts accommodate patents that may be associated with multiple countries and/or technological classes. The approach proportionally distributes each patent's contribution across its associated categories.

Define:
- $f^*_{ijt}$ as the sum of the products of country and CPC class contributions for the radical patents ($R$) for country $i$ in technology class $j$ at time $t$.
- $f_{ijt}$ similarly for non-radical patents ($N$).

The fractional contributions for radical and non-radical patents are calculated as:

$$
f^*_{ijt} = \sum_{p \in R} \left( c_{ip} \times d_{jp} \right)
$$
$$
f_{ijt} = \sum_{p \in N} \left( c_{ip} \times d_{jp} \right)
$$

where $c_{ip}$ denotes the fractional contribution of patent $p$ to country $i$, and $d_{jp}$ denotes the fractional contribution of patent $p$ to CPC class $j$.


The Radicality Index (RI) is then calculated as:
$$
RI_{ijt} = \frac{f^*_{ijt}}{f_{ijt} + f^*_{ijt}}
$$

This formula measures the share of radical innovations relative to the total innovations in a particular sector and year, indicating areas of comparative advantage or specialization. Normalizing the measure: 

$$
RI' = \frac{RI_{ijt} - 1}{RI_{ijt} + 1}
$$

Normalization scales the RI between -1 and 1, where positive values indicate a comparative advantage in radical innovations relative to the total innovations in a specific sector and year.

In [6]:
def expand_dict_rows(df, col_name):
    """ Normalize dictionary columns and expand them into rows. """
    rows = []
    for index, row in df.iterrows():
        dict_data = row[col_name]
        if isinstance(dict_data, str):
            dict_data = ast.literal_eval(dict_data)  # evaluate string literal to dictionary
        if isinstance(dict_data, dict):
            for key, value in dict_data.items():
                new_row = row.to_dict()
                new_row['key'] = key
                new_row['value'] = value
                rows.append(new_row)
    return pd.DataFrame(rows)

def calculate_fractional_index(df):
    # expand dictionary data to rows
    country_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'inventors_country_frac']], 'inventors_country_frac')
    class_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'frac_class']], 'frac_class')

    # merge and calculate weighted contributions
    merged_df = pd.merge(country_frac, class_frac, on=['patent', 'is_radical', 'year'], suffixes=('_country', '_class'))
    merged_df['weighted_contribution'] = merged_df['value_country'] * merged_df['value_class']

    # group by radical status and aggregate contributions
    grouped = merged_df.groupby(['key_country', 'key_class', 'year', 'is_radical'])
    sum_data = grouped['weighted_contribution'].sum().unstack(fill_value=0)

    # rename columns for clarity
    sum_data.rename(columns={0: 'non_radical_contributions', 1: 'radical_contributions'}, inplace=True)

    # calculate the RI and normalize it
    sum_data['RI'] = sum_data['radical_contributions'] / (sum_data['non_radical_contributions'] + sum_data['radical_contributions'])
    sum_data['RI_normalized'] = (sum_data['RI'] - 1) / (sum_data['RI'] + 1)

    return sum_data.reset_index()

result_df = calculate_fractional_index(df)
result_df

is_radical,key_country,key_class,year,non_radical_contributions,radical_contributions,RI,RI_normalized
0,AT,B01D,1992,0.20,0.00,0.000000,-1.000000
1,AT,C02F,1992,0.20,0.00,0.000000,-1.000000
2,AT,C07D,1992,0.20,0.00,0.000000,-1.000000
3,AT,D01F,1992,0.20,0.00,0.000000,-1.000000
4,AT,Y02P,1992,0.20,0.00,0.000000,-1.000000
...,...,...,...,...,...,...,...
729,US,Y10T,1995,0.50,0.00,0.000000,-1.000000
730,US,Y10T,1996,1.16,0.83,0.417085,-0.411348
731,US,Y10T,1997,1.82,0.50,0.215517,-0.645390
732,US,Y10T,1999,1.83,0.50,0.214592,-0.646643


In [ ]:
inst_dict = {'AU':'LME','CA':'LME','GB':'LME','IE':'LME','NZ':'LME','US':'LME',
                  'AT':'CME','BE':'CME','CH':'CME','DE':'CME','DT':'CME','DK':'CME','FI':'CME',
                  'JP':'CME', 'JA':'CME','KR':'CME','NL':'CME','NO':'CME','SE':'CME',
                'PT':'MME', 'ES':'MME','FR':'MME', 'IT':'MME', 'GR':'MME'}


finaldf['assignee_inst']= finaldf['assignee_country'].map(inst_dict).fillna("other")
finaldf['inventor_inst']= finaldf['inventor_country'].map(inst_dict).fillna("other")

<h3>Using Fractional Counts to Calculate a Balassa Index of Innovation Advantage</h3>

Fractional counts accommodate patents that may be associated with multiple countries and/or technological classes. The approach proportionally distributes each patent's contribution across its associated categories.

Let:
- $f^*_{ijt}$ represent the fractional count of radical patents for country $i$ in technology class $j$ at time $t$, calculated as the sum of the product of country and class contributions for radical patents.
- $f_{ijt}$ represent the fractional count of non-radical patents for the same parameters, calculated similarly.

These fractional counts are derived from expanding dictionary data within each patent record, which specify the contributions of each patent to various countries and CPC classes.

When using fractional counts for country and CPC class, the formula for the Radicality Index (RI) remains the same. The notation simply reflects the fractional nature of the counts. The Radicality Index effectively utilizes fractional counts to accommodate patents that may be associated with multiple countries or technological classes. 

$$
\large RI_{ijt} = \frac{\frac{f^*_{ijt}}{f_{ijt} + f^*_{ijt}}}{\sum_{i=1}^{n} \frac{f^*_{ijt}}{f_{ijt} + f^*_{ijt}}}
$$

The numerator calculates the proportion of radical patents, weighted by their fractional count, to the total patents (both radical and non-radical, each weighted by their fractional count) for each country in a specific sector and year.

The denominator aggregates these ratios over all countries for sector $j$ at time $t$.


In [ ]:
import pandas as pd

def expand_dict_rows(df, col_name):
    """ Normalize dictionary columns and expand them into rows. """
    rows = []
    for index, row in df.iterrows():
        dict_data = row[col_name]
        if isinstance(dict_data, str):
            dict_data = ast.literal_eval(dict_data)  # Safely evaluate string literal to dictionary
        if isinstance(dict_data, dict):
            for key, value in dict_data.items():
                new_row = row.to_dict()
                new_row['key'] = key
                new_row['value'] = value
                rows.append(new_row)
    return pd.DataFrame(rows)

def calculate_fractional_index(df):
    if 'is_radical' not in df.columns or 'year' not in df.columns:
        raise ValueError("DataFrame does not contain necessary columns ('is_radical' or 'year').")

    country_frac = expand_dict_rows(df, 'inventors_country_frac')
    class_frac = expand_dict_rows(df, 'frac_class')

    merged_df = pd.merge(country_frac, class_frac, on=['patent', 'is_radical', 'year'], suffixes=('_country', '_class'))
    merged_df['weighted_is_radical'] = merged_df['value_country'] * merged_df['value_class'] * merged_df['is_radical'].astype(int)
    merged_df['weighted_total'] = merged_df['value_country'] * merged_df['value_class']

    # Convert 'value_country' and 'value_class' to numeric if not already
    merged_df['value_country'] = pd.to_numeric(merged_df['value_country'], errors='coerce')
    merged_df['value_class'] = pd.to_numeric(merged_df['value_class'], errors='coerce')

    grouped = merged_df.groupby(['key_country', 'key_class', 'year']).agg({
        'weighted_is_radical': 'sum',
        'weighted_total': 'sum'
    }).reset_index()

    # Calculate the index based on weighted radical and total patents
    grouped['radicality_index'] = grouped['weighted_is_radical'] / grouped['weighted_total']

    # Normalize the radicality index using the specified formula
    grouped['radicality_index_normalized'] = (grouped['radicality_index'] - 1) / (grouped['radicality_index'] + 1)

    return grouped

# Assuming df is your DataFrame containing the patent data
result_df = calculate_fractional_index(df)
print(result_df)


In [ ]:
def expand_dict_rows(df, col_name):
    """ Helper function to normalize dictionary columns and expand them into rows. """
    rows = []
    for i, data in df.iterrows():
        # Convert string to dictionary if necessary
        dict_data = data[col_name]
        if isinstance(dict_data, str):
            try:
                dict_data = ast.literal_eval(dict_data)
            except ValueError:
                continue  # skip rows where the dictionary is malformed

        # proceed if dict_data is a dictionary
        if isinstance(dict_data, dict):
            for key, value in dict_data.items():
                new_data = data.to_dict()
                new_data['key'] = key
                new_data['value'] = value
                rows.append(new_data)
    new_df = pd.DataFrame(rows)
    return new_df

def calculate_fractional_index(df):
    if 'is_radical' not in df.columns or 'year' not in df.columns:
        raise ValueError("DataFrame does not contain necessary columns ('is_radical' or 'year').")

    # include 'year' in the selection
    country_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'inventors_country_frac']], 'inventors_country_frac')
    class_frac = expand_dict_rows(df[['patent', 'is_radical', 'year', 'frac_class']], 'frac_class')

    # merge expanded frames on patent, is_radical, and year to align country fractions with class fractions
    merged_df = pd.merge(country_frac, class_frac, on=['patent', 'is_radical', 'year'], suffixes=('_country', '_class'))

    # calculate the product of country and class fractions as the weight for each patent
    merged_df['weighted_is_radical'] = merged_df['value_country'] * merged_df['value_class'] * merged_df['is_radical'].astype(int)
    merged_df['weighted_total'] = merged_df['value_country'] * merged_df['value_class']

    # group by country, class, and year to aggregate the weighted counts
    grouped = merged_df.groupby(['key_country', 'key_class', 'year'])
    sum_data = grouped.agg({
        'weighted_is_radical': 'sum',
        'weighted_total': 'sum'
    })

    # calculate the index based on weighted radical and total patents
    sum_data['radicality_index'] = sum_data['weighted_is_radical'] / sum_data['weighted_total']

    # normalize the radicality index
    sum_data['radicality_index_normalized'] = (sum_data['radicality_index'] - 1) / (sum_data['radicality_index'] + 1)

    return sum_data.reset_index()

result_df = calculate_fractional_index(df)
print(result_df)
